## Numerical integration of Ordinary Differential Equations
This notebook serves as a quick refresher on ordinary differntial equations. If you are familiar with the topic: feel free to skip this notebook.

We will use the decay of tritium as an example:

$$
\mathrm{^3H \overset{\lambda}\rightarrow\ ^3He + e^- + \bar{\nu_e}}
$$

We will not concern ourselves with the products, instead we will only take interest in the number density of $^3H$ as function of time ($y(t)$). The rate of change of $y(t)$ is proportional to itself and the decay constant ($\lambda$):

$$
\frac{dy}{dt} = -\lambda y
$$

you probably know the solution to this kind of differential equations (either from experience or by guessing an appropriate ansatz). SymPy can of course also solve this equation:

In [ ]:
import sympy as sym
sym.init_printing()
t, l = sym.symbols('t lambda')
y = sym.Function('y')(t)
dydt = y.diff(t)
expr = sym.Eq(dydt, -l*y)
expr

In [ ]:
sym.dsolve(expr)

Now, pretend for a while that this function lacked an analytic solution. We could then integrate this equation numerically from an initial state for a predetermined amount of time by discretizing the time into a seriers of small steps. And for each of these steps we would update $y$ by multiplying the derivative with the step size (assuming that the derivate is approximately constant on the scale of the step-size), formally this method is known as "forward Euler":

$$
y_{n+1} = y_n + y'_n(t_n)\cdot \Delta h
$$

In [ ]:
def euler_fw(rhs, y0, tout, params):
    yout = np.zeros((len(tout), 1))
    yout[0] = y0
    t_old = tout[0]
    for i, t in enumerate(tout[1:], 1):
        dydt = rhs(yout[i-1], t, *params)
        h = t - t_old
        yout[i] = yout[i-1] + dydt*h
        t_old = t
    return yout

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
tout = np.linspace(0, 1, 100)
yout = euler_fw(lambda y, t, l: -l*y, 3, tout, (2,))
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(tout, yout[:, 0], tout, 3*np.exp(-2*tout))
axes[1].plot(tout, yout[:, 0]   -    3*np.exp(-2*tout))

Well, that was easy: 100 points gave us almost plotting accuracy. Unfortunately, Euler forward is not practical for most real world problems. Usually we want a higher order formula (the error in euler forward scales only as $n^{-1}$), and we want to use an adaptive step size (larger steps when the function is smooth). Furthermore, for a large class of problems, we need to base the step not on the derivative at the current time point, but rather at the next time point (giving rise to an implicit expression). We will not go into the details of this at all. Instead we will use the well tested LSODA algorithm (provided in scipy as ``odeint``):

In [ ]:
from scipy.integrate import odeint
yout, info = odeint(lambda y, t, l: -l*y, 3, tout, (2,), full_output=True)
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(tout, yout[:, 0], tout, 3*np.exp(-2*tout))
axes[1].plot(tout, yout[:, 0]   -    3*np.exp(-2*tout))
print("Number of function evaluations: %d" % info['nfe'][-1])

We can see that ``odeint`` was able to achieve a much higher precision using fewer number of function evaluations. Furthermore, the LSODA algortihm switches to an implicit stepper if the problem becomes "stiff".

In the upcoming notebooks we will use ``odeint`` to solve systems of ODEs (and not only linear equations as in this notebook). The emphasis is not on the numerical methods, but rather on how we, from symbolic expressions, can generate fast functions for the solver.

More information about the options we may pass to odeint is available using the ``help`` command: (or a ? in ipython)

In [ ]:
help(odeint)